<a href="https://colab.research.google.com/github/danielrjakab1/bdb_reaccuracy/blob/main/TrainingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import math
import pandas as pd

In [ ]:
# mount drive to access .csv files
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
base_path = '/content/drive/Shareddrives/BDB Reacuuracy/train/'

inputs = [pd.read_csv(f'{base_path}input_2023_w{str(i).zfill(2)}.csv') for i in range(1, 18)]
inputs_dict = {f'w{str(i).zfill(2)}': df for i, df in enumerate(inputs, start=1)}

supplementary_path = '/content/drive/Shareddrives/BDB Reacuuracy/supplementary/supplementary_data.csv'
supplementary = pd.read_csv(supplementary_path)

print(supplementary.columns)

Index(['game_id', 'season', 'week', 'game_date', 'game_time_eastern',
       'home_team_abbr', 'visitor_team_abbr', 'play_id', 'play_description',
       'quarter', 'game_clock', 'down', 'yards_to_go', 'possession_team',
       'defensive_team', 'yardline_side', 'yardline_number',
       'pre_snap_home_score', 'pre_snap_visitor_score',
       'play_nullified_by_penalty', 'pass_result', 'pass_length',
       'offense_formation', 'receiver_alignment', 'route_of_targeted_receiver',
       'play_action', 'dropback_type', 'dropback_distance',
       'pass_location_type', 'defenders_in_the_box', 'team_coverage_man_zone',
       'team_coverage_type', 'penalty_yards', 'pre_penalty_yards_gained',
       'yards_gained', 'expected_points', 'expected_points_added',
       'pre_snap_home_team_win_probability',
       'pre_snap_visitor_team_win_probability',
       'home_team_win_probability_added', 'visitor_team_win_probility_added'],
      dtype='object')


/tmp/ipython-input-388223751.py:7: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  supplementary = pd.read_csv(supplementary_path)


In [ ]:
print(supplementary['dropback_type'].dropna().unique())

['TRADITIONAL' 'DESIGNED_ROLLOUT_RIGHT' 'SCRAMBLE'
 'SCRAMBLE_ROLLOUT_RIGHT' 'DESIGNED_ROLLOUT_LEFT' 'SCRAMBLE_ROLLOUT_LEFT']


In [ ]:
print(supplementary['team_coverage_man_zone'].dropna().unique())

['ZONE_COVERAGE' 'MAN_COVERAGE']


In [ ]:
for week, df in inputs_dict.items():
    print(f'{week} input columns:', list(df.columns))

w01 input columns: ['game_id', 'play_id', 'player_to_predict', 'nfl_id', 'frame_id', 'play_direction', 'absolute_yardline_number', 'player_name', 'player_height', 'player_weight', 'player_birth_date', 'player_position', 'player_side', 'player_role', 'x', 'y', 's', 'a', 'dir', 'o', 'num_frames_output', 'ball_land_x', 'ball_land_y']
w02 input columns: ['game_id', 'play_id', 'player_to_predict', 'nfl_id', 'frame_id', 'play_direction', 'absolute_yardline_number', 'player_name', 'player_height', 'player_weight', 'player_birth_date', 'player_position', 'player_side', 'player_role', 'x', 'y', 's', 'a', 'dir', 'o', 'num_frames_output', 'ball_land_x', 'ball_land_y']
w03 input columns: ['game_id', 'play_id', 'player_to_predict', 'nfl_id', 'frame_id', 'play_direction', 'absolute_yardline_number', 'player_name', 'player_height', 'player_weight', 'player_birth_date', 'player_position', 'player_side', 'player_role', 'x', 'y', 's', 'a', 'dir', 'o', 'num_frames_output', 'ball_land_x', 'ball_land_y']
w

In [ ]:
inputs_dict['w01'].head()

,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,...,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
0,2023090700,101,False,54527,1,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.94,0.09,0.39,322.40,238.24,21,63.259998,-0.22
1,2023090700,101,False,54527,2,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.94,0.04,0.61,200.89,236.05,21,63.259998,-0.22
2,2023090700,101,False,54527,3,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.93,0.12,0.73,147.55,240.60,21,63.259998,-0.22
3,2023090700,101,False,54527,4,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.35,36.92,0.23,0.81,131.40,244.25,21,63.259998,-0.22
4,2023090700,101,False,54527,5,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.37,36.90,0.35,0.82,123.26,244.25,21,63.259998,-0.22


In [ ]:
all_roles = set()

for df in inputs_dict.values():
    if 'player_role' in df.columns:
        all_roles.update(df['player_role'].dropna().unique())

print(sorted(all_roles))

['Defensive Coverage', 'Other Route Runner', 'Passer', 'Targeted Receiver']


In [ ]:
all_roles = set()

for df in inputs_dict.values():
    if 'player_position' in df.columns:
        all_roles.update(df['player_position'].dropna().unique())

print(sorted(all_roles))

['CB', 'DE', 'DT', 'FB', 'FS', 'ILB', 'LB', 'MLB', 'NT', 'OLB', 'QB', 'RB', 'S', 'SS', 'T', 'TE', 'WR']


In [ ]:
for week, df in inputs_dict.items():
    print(f"{week} has {df.shape[0]} rows")

w01 has 285714 rows
w02 has 288586 rows
w03 has 297757 rows
w04 has 272475 rows
w05 has 254779 rows
w06 has 270676 rows
w07 has 233597 rows
w08 has 281011 rows
w09 has 252796 rows
w10 has 260372 rows
w11 has 243413 rows
w12 has 294940 rows
w13 has 233755 rows
w14 has 279972 rows
w15 has 281820 rows
w16 has 316417 rows
w17 has 277582 rows


In [ ]:
cleaned_inputs = {}
diagnostics = {}

DL_POSITIONS = ['DL','DE','DT','NT','EDGE']

for week, df in inputs_dict.items():
    df = df.copy()
    before = len(df)  # number of rows before cleaning

    # Merge supplementary pass_result
    df = df.merge(
        supplementary[['game_id','play_id','pass_result', 'team_coverage_man_zone']],
        on=['game_id','play_id'],
        how='left'
    )

    # Keep only pass plays
    df = df[df['pass_result'].isin(['C','I','IN'])]

    # --- NEW: Filter down to only MAN coverage plays BEFORE finding primary defender ---
    df_man = df[df['team_coverage_man_zone'] == 'MAN_COVERAGE'].copy()

    # Identify primary defender
    target_df = df_man.merge(
        df_man[df_man['player_role'] == 'Targeted Receiver'][['game_id','play_id','x','y']],
        on=['game_id','play_id'],
        suffixes=('','_target')
    )
    target_df['dist_to_target'] = np.sqrt(
        (target_df['x'] - target_df['x_target'])**2 +
        (target_df['y'] - target_df['y_target'])**2
    )
    primary_defenders = target_df.groupby(['game_id','play_id'])['dist_to_target'].idxmin()
    print("Number of primary defenders identified:", len(primary_defenders))
    df = df.drop(primary_defenders, errors='ignore')

    # Keep only Defensive Coverage
    df = df[df['player_role'] == 'Defensive Coverage']

    # Keep only defensive players
    df = df[df['player_side'] == 'Defense']

    # Remove defensive linemen
    df = df[~df['player_position'].isin(DL_POSITIONS)]

    # Store cleaned data
    cleaned_inputs[week] = df
    after = len(df)

    # Diagnostics
    diagnostics[week] = {
        'rows_before': before,
        'rows_after': after,
        'rows_removed': before - after,
        'unique_roles_remaining': sorted(df['player_role'].dropna().unique().tolist()),
        'unique_positions_remaining': sorted(df['player_position'].dropna().unique().tolist())
    }

Number of primary defenders identified: 225
Number of primary defenders identified: 219
Number of primary defenders identified: 293
Number of primary defenders identified: 196
Number of primary defenders identified: 219
Number of primary defenders identified: 198
Number of primary defenders identified: 196
Number of primary defenders identified: 255
Number of primary defenders identified: 198
Number of primary defenders identified: 222
Number of primary defenders identified: 191
Number of primary defenders identified: 273
Number of primary defenders identified: 203
Number of primary defenders identified: 222
Number of primary defenders identified: 210
Number of primary defenders identified: 263
Number of primary defenders identified: 235


In [ ]:
for week, d in diagnostics.items():
    print(f"{week}: before={d['rows_before']}, after={d['rows_after']}, removed={d['rows_removed']}")
    print(f"   roles remaining: {d['unique_roles_remaining']}")
    print(f"   positions remaining: {d['unique_positions_remaining']}\n")

w01: before=285714, after=153859, removed=131855
   roles remaining: ['Defensive Coverage']
   positions remaining: ['CB', 'FS', 'ILB', 'MLB', 'OLB', 'S', 'SS']

w02: before=288586, after=155798, removed=132788
   roles remaining: ['Defensive Coverage']
   positions remaining: ['CB', 'FS', 'ILB', 'MLB', 'OLB', 'S', 'SS']

w03: before=297757, after=160780, removed=136977
   roles remaining: ['Defensive Coverage']
   positions remaining: ['CB', 'FS', 'ILB', 'MLB', 'OLB', 'S', 'SS']

w04: before=272475, after=147827, removed=124648
   roles remaining: ['Defensive Coverage']
   positions remaining: ['CB', 'FS', 'ILB', 'MLB', 'OLB', 'S', 'SS']

w05: before=254779, after=138283, removed=116496
   roles remaining: ['Defensive Coverage']
   positions remaining: ['CB', 'FS', 'ILB', 'MLB', 'OLB', 'S', 'SS']

w06: before=270676, after=145783, removed=124893
   roles remaining: ['Defensive Coverage']
   positions remaining: ['CB', 'FS', 'ILB', 'MLB', 'OLB', 'S', 'SS']

w07: before=233597, after=12

In [ ]:
import os

output_folder = '/content/drive/Shareddrives/BDB Reacuuracy/cleaned_data'
os.makedirs(output_folder, exist_ok=True)

for week, df in cleaned_inputs.items():
    df.to_csv(f'{output_folder}/cleaned_input_{week}.csv', index=False)